### Word2vec  

En las secciones previas se explicó cómo representar palabras mediante vectores dispersos y de alta dimensión. Ahora se introduce una representación más eficaz: los *embeddings*, que son vectores densos y de baja dimensión (entre 50 y 1000 dimensiones). A diferencia de los vectores dispersos, los *embeddings* son más eficientes, mejoran la generalización de los modelos y capturan mejor relaciones como la sinonimia.

Se presenta el método **skip-gram** con **negative sampling** (SGNS), parte de **word2vec**, como una técnica para calcular *embeddings*. **word2vec** es rápido, eficiente para entrenar y ofrece *embeddings* preentrenados. Estos *embeddings* son estáticos, asignando un vector fijo a cada palabra, a diferencia de los *embeddings* contextuales dinámicos como los de **BERT**, que varían según el contexto.

La idea clave de **word2vec** es entrenar un clasificador para predecir si una palabra aparece cerca de otra, utilizando texto en ejecución como datos de entrenamiento de forma auto-supervisada. Esto elimina la necesidad de etiquetas manuales. A diferencia de los modelos de lenguaje neuronal más complejos, **word2vec** simplifica tanto la tarea (clasificación binaria) como la arquitectura (regresión logística en lugar de redes neuronales multicapa).

Por ejemplo la intuición de **skip-gram** es:

1. Tratar la palabra objetivo y una palabra de contexto vecina como ejemplos positivos.
2. Muestrear aleatoriamente otras palabras en el léxico para obtener ejemplos negativos.
3. Usar regresión logística para entrenar un clasificador que distinga esos dos casos.
4. Usar los pesos aprendidos como los *embeddings*.




#### 1. Representación de palabras como vectores densos

**Ejemplo:**

Supongamos que tenemos un pequeño vocabulario compuesto por las siguientes palabras: *manzana*, *naranja*, *fruta*, *coche*, *automóvil*, *vehículo*. Utilizando **word2vec**, cada una de estas palabras se representaría mediante un vector denso de, por ejemplo, 100 dimensiones. Aunque cada vector tiene 100 valores, no hay una interpretación directa de cada dimensión; sin embargo, las relaciones semánticas entre palabras se capturan en la proximidad de sus vectores en el espacio.

- **manzana**: `[0.25, -0.10, ..., 0.47]`
- **naranja**: `[0.30, -0.08, ..., 0.50]`
- **fruta**: `[0.28, -0.12, ..., 0.45]`
- **coche**: `[0.60, 0.15, ..., -0.20]`
- **automóvil**: `[0.62, 0.18, ..., -0.22]`
- **vehículo**: `[0.59, 0.14, ..., -0.19]`

#### 2. Captura de sinonimia y relaciones semánticas

**Ejemplo:**

Observemos que *coche*, *automóvil* y *vehículo* están relacionadas semánticamente. En el espacio de embeddings, estos vectores estarán cerca entre sí, reflejando su sinonimia y relación temática. Por otro lado, *manzana*, *naranja* y *fruta* también estarán agrupadas, pero separadas de las palabras relacionadas con vehículos.

Además, **word2vec** puede capturar relaciones más complejas. Por ejemplo:

- **Reina** - **Hombre** + **Mujer** ≈ **Reina**

Esto significa que si restamos el vector de *hombre* del de *rey* y sumamos el vector de *mujer*, obtenemos un vector que está cerca del vector de *reina*.

#### 3. Funcionamiento del modelo Skip-Gram con negative sampling (SGNS)

**Ejemplo:**

Supongamos que queremos entrenar un modelo **skip-gram** para la palabra objetivo *fruta*. El modelo intentará predecir las palabras de contexto que aparecen cerca de *fruta* en el texto.

- **Contexto positivo:** Si en el corpus aparece la frase "La **fruta** es saludable", las palabras *la*, *es*, *saludable* serán ejemplos positivos.

- **Contexto negativo:** El modelo también seleccionará aleatoriamente palabras que no aparecen cerca de *fruta* en ese contexto, por ejemplo, *coche*, *automóvil*, *vehículo*, para actuar como ejemplos negativos.

El modelo entrenará un clasificador que aprende a distinguir entre las palabras que realmente aparecen en el contexto de *fruta* y las que no, ajustando los vectores de embeddings en consecuencia.

#### 4. Auto-supervisión en el entrenamiento de embeddings

**Ejemplo:**

Consideremos una oración del corpus: "El **coche** rojo acelera por la **carretera**".

Durante el entrenamiento con **word2vec**, el modelo utiliza esta oración para generar ejemplos de entrenamiento de forma automática:

- **Palabra objetivo:** *coche*
  - **Contexto:** *el*, *rojo*, *acelera*, *por*, *la*, *carretera*

El modelo crea pares positivos (*coche*, *el*), (*coche*, *rojo*), etc., y genera pares negativos seleccionando palabras al azar del vocabulario que no aparecen en el contexto de *coche*. Así, utiliza el texto existente como señal de supervisión sin necesidad de etiquetas manuales.

#### 5. Comparación entre embeddings estáticos y contextuales

**Ejemplo:**

- **Embeddings estáticos (word2vec):**
  - La palabra *banco* tendrá un único vector de embedding, independientemente del contexto en el que aparezca.
  - Por ejemplo:
    - *banco* (institución financiera): `[0.45, -0.22, ..., 0.33]`
    - *banco* (asiento): `[0.45, -0.22, ..., 0.33]`

- **Embeddings contextuales (BERT):**
  - La palabra *banco* tendrá diferentes vectores dependiendo de su contexto.
  - Por ejemplo:
    - En "Voy al **banco** a sacar dinero": *banco* podría tener un vector cercano al de *institución financiera*.
    - En "Me senté en el **banco** del parque": *banco* podría tener un vector cercano al de *asiento*.

#### 6. Ventajas de usar vectores densos sobre vectores dispersos

**Ejemplo:**

Imaginemos que tenemos un vocabulario de 50,000 palabras. Representar cada palabra como un vector disperso implicaría vectores de 50,000 dimensiones, la mayoría de los cuales serían ceros. Esto no solo consume mucha memoria, sino que también dificulta el aprendizaje de modelos debido a la alta dimensionalidad.

En cambio, usando **word2vec** para representar cada palabra con un vector denso de 300 dimensiones:

- **Espacio de parámetros reducido:** Menos dimensiones significan que el clasificador necesita aprender menos pesos, lo que facilita la generalización y reduce el riesgo de sobreajuste.
  
- **Captura de similitud semántica:** Los vectores densos permiten que palabras similares estén cerca en el espacio vectorial, mejorando tareas como la clasificación de texto, la traducción automática y la detección de sinónimos.



In [1]:
#  Representación de palabras como vectores densos
import numpy as np

# Definir un vocabulario pequeño
vocabulario = ['manzana', 'naranja', 'fruta', 'coche', 'automóvil', 'vehículo']

# Dimensionalidad de los embeddings
d = 100

# Inicializar embeddings aleatorios para cada palabra
embeddings = {}
np.random.seed(42)  # Para reproducibilidad
for palabra in vocabulario:
    embeddings[palabra] = np.random.randn(d)

# Mostrar los embeddings
for palabra, vector in embeddings.items():
    print(f"{palabra}: {vector[:5]}...")  # Mostrar solo las primeras 5 dimensiones


manzana: [ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337]...
naranja: [-1.41537074 -0.42064532 -0.34271452 -0.80227727 -0.16128571]...
fruta: [ 0.35778736  0.56078453  1.08305124  1.05380205 -1.37766937]...
coche: [-0.82899501 -0.56018104  0.74729361  0.61037027 -0.02090159]...
automóvil: [-1.59442766 -0.59937502  0.0052437   0.04698059 -0.45006547]...
vehículo: [ 0.92617755  1.90941664 -1.39856757  0.56296924 -0.65064257]...


In [2]:
#  Captura de sinonimia y relaciones semánticas
from numpy.linalg import norm

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

# Calcular similitudes
sim_coche_automovil = cos_sim(embeddings['coche'], embeddings['automóvil'])
sim_coche_vehiculo = cos_sim(embeddings['coche'], embeddings['vehículo'])
sim_manzana_naranja = cos_sim(embeddings['manzana'], embeddings['naranja'])
sim_reina_hombre = 0  # Placeholder
sim_rey_mujer = 0    # Placeholder

print(f"Similitud entre 'coche' y 'automóvil': {sim_coche_automovil:.4f}")
print(f"Similitud entre 'coche' y 'vehículo': {sim_coche_vehiculo:.4f}")
print(f"Similitud entre 'manzana' y 'naranja': {sim_manzana_naranja:.4f}")


Similitud entre 'coche' y 'automóvil': 0.2060
Similitud entre 'coche' y 'vehículo': 0.0565
Similitud entre 'manzana' y 'naranja': -0.1382


In [3]:
# Funcionamiento del modelo Skip-Gram con negative sampling (SGNS)
import numpy as np
import random

# Parámetros
d = 50  # Dimensionalidad de los embeddings
window_size = 2 
negative_samples = 2
epochs = 1000
learning_rate = 0.01

# Corpus de ejemplo
corpus = [
    "la manzana es saludable",
    "el coche rojo acelera",
    "la naranja es jugosa",
    "el automóvil es rápido",
    "la fruta es deliciosa",
    "el vehículo avanza por la carretera"
]

# Preprocesar el corpus: convertir a minúsculas y tokenizar
preprocesado = [line.lower().split() for line in corpus]

# Construir el vocabulario dinámicamente a partir del corpus
vocabulario = list(set([palabra for frase in preprocesado for palabra in frase]))
print(f"Vocabulario ({len(vocabulario)} palabras): {vocabulario}")

# Crear índices para las palabras
word_to_idx = {word: idx for idx, word in enumerate(vocabulario)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Inicializar matrices de embeddings (input y output) con valores pequeños
np.random.seed(42)  # Para reproducibilidad
W_in = np.random.randn(len(vocabulario), d) * 0.01
W_out = np.random.randn(len(vocabulario), d) * 0.01

# Función de entrenamiento simplificada
def train_skipgram(corpus, W_in, W_out, epochs=1000, learning_rate=0.01):
    for epoch in range(1, epochs + 1):
        for sentence in corpus:
            for i, palabra in enumerate(sentence):
                target = word_to_idx[palabra]
                # Definir el contexto dentro del window_size
                context_indices = list(range(max(0, i - window_size), min(len(sentence), i + window_size + 1)))
                context_indices.remove(i)  # Remover el target del contexto
                for j in context_indices:
                    context = word_to_idx[sentence[j]]
                    
                    # Positivo
                    z = np.dot(W_out[context], W_in[target])
                    sigmoid = 1 / (1 + np.exp(-z))
                    error = 1 - sigmoid
                    # Actualizar W_out y W_in para el ejemplo positivo
                    W_out[context] += learning_rate * error * W_in[target]
                    W_in[target] += learning_rate * error * W_out[context]
                    
                    # Negativos
                    for _ in range(negative_samples):
                        neg = random.randint(0, len(vocabulario)-1)
                        while neg == context:
                            neg = random.randint(0, len(vocabulario)-1)
                        z_neg = np.dot(W_out[neg], W_in[target])
                        sigmoid_neg = 1 / (1 + np.exp(-z_neg))
                        error_neg = 0 - sigmoid_neg
                        # Actualizar W_out y W_in para el ejemplo negativo
                        W_out[neg] += learning_rate * error_neg * W_in[target]
                        W_in[target] += learning_rate * error_neg * W_out[neg]
        
        # Mostrar progreso cada 200 epochs
        if epoch % 200 == 0 or epoch == 1 or epoch == epochs:
            print(f"Epoca {epoch} completada")
    
    return W_in, W_out

# Entrenar el modelo
W_in_trained, W_out_trained = train_skipgram(preprocesado, W_in, W_out, epochs, learning_rate)

# Obtener el embedding de una palabra
def get_embedding(word):
    if word in word_to_idx:
        return W_in_trained[word_to_idx[word]]
    else:
        raise ValueError(f"La palabra '{word}' no está en el vocabulario.")

# Mostrar el embedding de 'coche'
embedding_coche = get_embedding('coche')
print(f"Embedding de 'coche': {embedding_coche[:5]}...")  # Mostrar solo las primeras 5 dimensiones


Vocabulario (18 palabras): ['saludable', 'el', 'jugosa', 'fruta', 'deliciosa', 'avanza', 'es', 'acelera', 'automóvil', 'naranja', 'vehículo', 'rojo', 'por', 'rápido', 'carretera', 'manzana', 'coche', 'la']
Epoca 1 completada
Epoca 200 completada
Epoca 400 completada
Epoca 600 completada
Epoca 800 completada
Epoca 1000 completada
Embedding de 'coche': [ 0.12239252  0.36296517 -0.17998622  0.63964944 -0.51743025]...


In [4]:
# Auto-supervisión en el entrenamiento de embedding
import numpy as np
import random

# Definir un corpus de ejemplo
corpus = [
    "El coche rojo acelera por la carretera",
    "La manzana es saludable y la naranja es jugosa",
    "El automóvil rápido pasa al vehículo lento",
    "La fruta deliciosa se vende en el mercado",
    "El banco está cerrado hoy",
    "Me senté en el banco del parque"
]

# Preprocesar el corpus
preprocesado = [line.lower().split() for line in corpus]

# Construir el vocabulario
vocabulario = list(set([palabra for frase in preprocesado for palabra in frase]))
word_to_idx = {word: idx for idx, word in enumerate(vocabulario)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Parámetros
window_size = 2
negative_samples = 2
d = 50  # Dimensionalidad
learning_rate = 0.01
epochs = 1000

# Inicializar embeddings
W_in = np.random.randn(len(vocabulario), d)
W_out = np.random.randn(len(vocabulario), d)

# Función de entrenamiento
def train_auto_supervision(corpus, W_in, W_out, epochs, learning_rate):
    for epoch in range(epochs):
        for sentence in corpus:
            for i, palabra in enumerate(sentence):
                target = word_to_idx[palabra]
                context_indices = list(range(max(0, i - window_size), min(len(sentence), i + window_size + 1)))
                context_indices.remove(i)
                for j in context_indices:
                    context = word_to_idx[sentence[j]]
                    
                    # Positivo
                    z = np.dot(W_out[context], W_in[target])
                    sigmoid = 1 / (1 + np.exp(-z))
                    error = 1 - sigmoid
                    W_out[context] += learning_rate * error * W_in[target]
                    W_in[target] += learning_rate * error * W_out[context]
                    
                    # Negativos
                    for _ in range(negative_samples):
                        neg = random.randint(0, len(vocabulario)-1)
                        while neg == context:
                            neg = random.randint(0, len(vocabulario)-1)
                        z_neg = np.dot(W_out[neg], W_in[target])
                        sigmoid_neg = 1 / (1 + np.exp(-z_neg))
                        error_neg = 0 - sigmoid_neg
                        W_out[neg] += learning_rate * error_neg * W_in[target]
                        W_in[target] += learning_rate * error_neg * W_out[neg]
        if epoch % 200 == 0:
            print(f"Epoca {epoch} completada")
    return W_in, W_out

# Entrenar el modelo
W_in_trained, W_out_trained = train_auto_supervision(preprocesado, W_in, W_out, epochs, learning_rate)

# Obtener el embedding de 'coche'
def get_embedding(word):
    return W_in_trained[word_to_idx[word]]

print(f"Embedding de 'coche': {get_embedding('coche')[:5]}...")


Epoca 0 completada
Epoca 200 completada
Epoca 400 completada
Epoca 600 completada
Epoca 800 completada
Embedding de 'coche': [-1.1343032   0.15760294 -0.67531012 -0.33490473  1.87541599]...


In [7]:
# Comparación entre embeddings estáticos y contextuales
import numpy as np

# Embeddings estáticos: un único vector por palabra
embeddings_estaticos = {
    'banco': np.random.randn(50)
}

# Embeddings contextuales: diferentes vectores según el contexto
# Por simplicidad, usaremos dos contextos diferentes
contexto_financiero = "Voy al banco a sacar dinero".lower().split()
contexto_asiento = "Me senté en el banco del parque".lower().split()

# Función para obtener embeddings contextuales
def obtener_embedding_contextual(palabra, contexto, embeddings):
    # Simplemente agregamos ruido basado en el contexto para ilustrar
    context_factor = 1 if 'dinero' in contexto else -1
    return embeddings_estaticos.get(palabra, np.random.randn(50)) + context_factor * 1.0

# Obtener embeddings
embedding_banco_fin = obtener_embedding_contextual('banco', contexto_financiero, embeddings_estaticos)
embedding_banco_asiento = obtener_embedding_contextual('banco', contexto_asiento, embeddings_estaticos)

# Comparar
similitud = np.dot(embedding_banco_fin, embedding_banco_asiento) / (norm(embedding_banco_fin) * norm(embedding_banco_asiento))
print(f"Similitud entre 'banco' en contexto financiero y de asiento: {similitud:.4f}")


Similitud entre 'banco' en contexto financiero y de asiento: 0.1388


In [18]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Cargar el tokenizador y el modelo preentrenado multilingüe de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Función para obtener el embedding contextual de una palabra en un contexto específico
def obtener_embedding_contextual(frase, palabra, tokenizer, model):
    # Tokenizar la frase
    inputs = tokenizer(frase, return_tensors='pt')
    outputs = model(**inputs)

    # Convertir la salida en embeddings (última capa de BERT)
    embeddings = outputs.last_hidden_state.squeeze(0).detach().numpy()
    
    # Tokenizar la frase para obtener la lista de tokens
    tokens = tokenizer.tokenize(frase)
    
    # Buscar todas las ocurrencias de la palabra en los tokens
    # En modelos multilingües, es posible que la palabra se divida en subpalabras
    word_tokens = tokenizer.tokenize(palabra)
    token_len = len(word_tokens)
    
    # Encontrar la posición de la palabra en los tokens
    posiciones = []
    for i in range(len(tokens) - token_len + 1):
        if tokens[i:i+token_len] == word_tokens:
            posiciones.append(i)
    
    if not posiciones:
        raise ValueError(f"La palabra '{palabra}' no fue encontrada en la frase.")
    
    # Asumimos que la palabra aparece una vez; para múltiples apariciones, puedes adaptar el código
    palabra_idx = posiciones[0]
    
    if token_len == 1:
        # Si la palabra no se dividió en subpalabras
        embedding = embeddings[palabra_idx]
    else:
        # Si la palabra se dividió en subpalabras, promediar sus embeddings
        embedding = np.mean(embeddings[palabra_idx:palabra_idx+token_len], axis=0)
    
    return embedding

# Contextos diferentes
contexto_financiero = "Voy al banco a sacar dinero"
contexto_asiento = "Me senté en el banco del parque"

# Obtener los embeddings contextuales de la palabra 'banco'
embedding_banco_fin = obtener_embedding_contextual(contexto_financiero, 'banco', tokenizer, model)
embedding_banco_asiento = obtener_embedding_contextual(contexto_asiento, 'banco', tokenizer, model)

# Calcular la similitud coseno entre los dos embeddings
def similitud_coseno(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similitud = similitud_coseno(embedding_banco_fin, embedding_banco_asiento)
print(f"Similitud entre 'banco' en contexto financiero y de asiento: {similitud:.4f}")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Similitud entre 'banco' en contexto financiero y de asiento: 0.6015


En un modelo real de embeddings contextuales como BERT, la similitud sería menor ya que los vectores reflejarían diferentes significados. En este ejemplo simplificado, hemos añadido un pequeño cambio para ilustrar la diferencia.

In [6]:
# Ventajas de usar vectores densos sobre vectores dispersos
import numpy as np

# Supongamos un vocabulario grande
vocabulario_grande = [f"palabra_{i}" for i in range(50000)]
d = 300  # Dimensionalidad de los embeddings densos

# Representación dispersa: vectores de 50,000 dimensiones con mayoría de ceros
# Ejemplo: Representación one-hot para una palabra
def one_hot(word, vocab):
    vector = np.zeros(len(vocab))
    vector[vocab.index(word)] = 1
    return vector

# Representación densa: vectores de 300 dimensiones
def dense_embedding(word, embeddings):
    return embeddings[word]

# Inicializar embeddings densos
embeddings_densos = {word: np.random.randn(d) for word in vocabulario_grande}

# Comparar el tamaño en memoria
import sys

# Tamaño de una representación dispersa para una palabra
palabra = 'palabra_12345'
vector_disperso = one_hot(palabra, vocabulario_grande)
tamaño_disperso = sys.getsizeof(vector_disperso)

# Tamaño de una representación densa para una palabra
vector_denso = dense_embedding(palabra, embeddings_densos)
tamaño_denso = sys.getsizeof(vector_denso)

print(f"Tamaño de vector disperso (one-hot) para una palabra: {tamaño_disperso} bytes")
print(f"Tamaño de vector denso para una palabra: {tamaño_denso} bytes")


Tamaño de vector disperso (one-hot) para una palabra: 400104 bytes
Tamaño de vector denso para una palabra: 2504 bytes


### Ejercicios

**Ejercicio 1: Comprensión de vectores densos**

**Descripción:**  

Explica la diferencia entre vectores dispersos y vectores densos en el contexto de la representación de palabras. ¿Por qué los vectores densos son preferidos en tareas de procesamiento del lenguaje natural?

**Puntos a considerar:**
- Dimensionalidad
- Interpretación de las dimensiones
- Eficiencia en el almacenamiento y procesamiento
- Capacidad para capturar relaciones semánticas

**Ejercicio 2: Captura de sinonimia y relaciones semánticas**

**Descripción:**  
Imagina que has entrenado un modelo **word2vec** con un corpus específico. Observas que las palabras *"gato"*, *"felino"* y *"minino"* tienen vectores muy similares entre sí, mientras que *"perro"* está algo separado pero aún cercano.  
- ¿Qué indica esta distribución sobre las relaciones semánticas entre estas palabras?
- ¿Cómo afectaría esto a tareas como la detección de sinónimos o la clasificación de texto?


**Ejercicio 3: Funcionamiento del modelo Skip-Gram con negative sampling (SGNS)**

**Descripción:**  
Describe paso a paso cómo el modelo **skip-gram** con **negative sampling** actualiza los vectores de embeddings durante el entrenamiento.  
- ¿Cuál es el propósito de los ejemplos negativos?
- ¿Cómo contribuyen los ejemplos positivos y negativos a la optimización de los embeddings?

---

**Ejercicio 4: Auto-Supervisión en el entrenamiento de embeddings**

**Descripción:**  
Explica el concepto de **auto-supervisión** en el contexto de **word2vec**.  
- ¿Cómo se generan las señales de supervisión de manera implícita a partir del texto?
- ¿Cuáles son las ventajas de utilizar auto-supervisión en comparación con métodos que requieren etiquetas manuales?

**Ejercicio 5: Embeddings estáticos vs. contextuales**

**Descripción:**  
Comparar y contrastar los **embeddings estáticos** (como los generados por **word2vec**) con los **embeddings contextuales** (como los de **BERT**).  
- ¿Cuáles son las principales diferencias en cómo representan las palabras?
- ¿En qué escenarios sería más beneficioso utilizar embeddings contextuales en lugar de estáticos?

**Ejercicio 6: Ventajas de usar vectores densos sobre vectores dispersos**

**Descripción:**  
Analiza las ventajas de utilizar representaciones densas de palabras frente a representaciones dispersas, especialmente en vocabularios grandes.  
- Considera aspectos como el uso de memoria, la eficiencia computacional y la capacidad de generalización.
- ¿Cómo impacta la dimensionalidad de los vectores en el rendimiento de los modelos de NLP?

**Ejercicio 7: Construcción del vocabulario dinámico**

**Descripción:**  
En el ejemplo corregido, se construyó el vocabulario dinámicamente a partir del corpus.  
- ¿Por qué es importante asegurarse de que todas las palabras del corpus estén incluidas en el vocabulario?
- ¿Qué problemas pueden surgir si se utilizan vocabularios estáticos que no abarcan todas las palabras del corpus?


**Ejercicio 8: Similitud coseno entre vectores de palabras**

**Descripción:**  
Después de entrenar un modelo **word2vec**, calculas la similitud coseno entre diferentes pares de palabras y observas que *"coche"* y *"automóvil"* tienen una alta similitud, mientras que *"coche"* y *"manzana"* tienen una baja similitud.  
- ¿Qué interpretaciones puedes hacer sobre las relaciones semánticas entre estas palabras?
- ¿Cómo podrías utilizar estas similitudes en aplicaciones prácticas de NLP?


**Ejercicio 9: Impacto de los hiperparámetros en el entrenamiento**

**Descripción:**  
Considera los hiperparámetros utilizados en el entrenamiento de **word2vec**, como `window_size`, `negative_samples`, `learning_rate` y `epochs`.  
- Explica cómo cada uno de estos hiperparámetros puede afectar la calidad de los embeddings resultantes.
- ¿Qué estrategias podrías emplear para seleccionar valores óptimos para estos hiperparámetros?


**Ejercicio 10: Evaluación de la calidad de los embeddings**

**Descripción:**  
Después de entrenar tus propios embeddings con **word2vec**, quieres evaluar su calidad.  
- Describe al menos dos métodos o tareas que podrías utilizar para evaluar la efectividad de tus embeddings.
- ¿Qué métricas específicas utilizarías para medir el rendimiento en estas tareas?


In [ ]:
## Tu respuesta

### El clasificador

En esta sección se explica cómo **skip-gram** utiliza un clasificador probabilístico para aprender *embeddings*. Consideremos una oración con una palabra objetivo, por ejemplo *apricot*, y una ventana de contexto de ±2 palabras. El objetivo es entrenar un clasificador que, dado un par $(w, c)$ donde $w$ es la palabra objetivo y $c$ una palabra candidata del contexto, calcule la probabilidad de que $c$ sea un contexto real de $w$, es decir, $P(+|w, c)$.

El clasificador basa esta probabilidad en la similitud de los *embeddings* de $w$ y $c$, calculada mediante el producto punto de sus vectores. Este valor se transforma en una probabilidad utilizando la función sigmoide $\sigma(x) = \frac{1}{1 + \exp(-x)}$. 

Así, $P(+|w, c) = \sigma(\mathbf{c} \cdot \mathbf{w})$ y $P(-|w, c) = 1 - P(+|w, c) = \sigma(-\mathbf{c} \cdot \mathbf{w})$.

**Skip-gram** asume que las palabras de contexto son independientes, permitiendo calcular la probabilidad conjunta como el producto de las probabilidades individuales: $P(+|w, c_1:L) = \prod_{i=1}^{L} \sigma(\mathbf{c_i} \cdot \mathbf{w})$. Al entrenar, se maximiza la suma de los logaritmos de estas probabilidades.

En resumen, **skip-gram** entrena un clasificador que asigna probabilidades basadas en la similitud de *embeddings* entre una palabra objetivo y sus contextos, utilizando la función sigmoide aplicada al producto punto de sus vectores. Este enfoque permite aprender representaciones densas y efectivas para cada palabra en el vocabulario.

In [ ]:
import numpy as np

# Definimos la función sigmoide σ(x) = 1 / (1 + exp(-x))
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Tamaño de los embeddings (dimensión de los vectores)
D = 5  # Puedes cambiar este valor

# Establecemos una semilla para reproducibilidad
np.random.seed(42)

# Creamos un embedding aleatorio para la palabra objetivo w (por ejemplo, 'apricot')
w_embedding = np.random.randn(D)

# Creamos embeddings aleatorios para las palabras de contexto c1, c2, c3, c4
c1_embedding = np.random.randn(D)
c2_embedding = np.random.randn(D)
c3_embedding = np.random.randn(D)
c4_embedding = np.random.randn(D)

# Agrupamos los embeddings de las palabras de contexto en una lista
context_embeddings = [c1_embedding, c2_embedding, c3_embedding, c4_embedding]

# Calculamos los productos punto c_i ⋅ w para cada palabra de contexto
# Esto corresponde a la ecuación: Similaridad(w, c) ≈ c ⋅ w
dot_products = [np.dot(c_embedding, w_embedding) for c_embedding in context_embeddings]

# Calculamos P(+|w, c_i) = σ(c_i ⋅ w) para cada palabra de contexto
# Esto corresponde a la ecuación: P(+|w, c) = σ(c ⋅ w)
P_positive = [sigmoid(dot_product) for dot_product in dot_products]

# Calculamos P(-|w, c_i) = σ(-c_i ⋅ w) para cada palabra de contexto
# Esto corresponde a la ecuación: P(-|w, c) = σ(-c ⋅ w)
P_negative = [sigmoid(-dot_product) for dot_product in dot_products]

# Alternativamente, podemos calcular P(-|w, c_i) = 1 - P(+|w, c_i)
# P(-|w, c_i) = 1 - P(+|w, c_i)
P_negative_alternative = [1 - p for p in P_positive]

# Verificamos que ambas formas de calcular P(-|w, c_i) son equivalentes
for i in range(len(P_negative)):
    assert np.isclose(P_negative[i], P_negative_alternative[i]), "Las probabilidades no coinciden"

# Suponiendo independencia, calculamos la probabilidad total P(+|w, c_1:L)
# Esto corresponde a la ecuación: P(+|w, c_1:L) = ∏_{i=1}^{L} σ(c_i ⋅ w)
P_positive_total = np.prod(P_positive)

# Calculamos el logaritmo de la probabilidad total
# Esto corresponde a la ecuación: log P(+|w, c_1:L) = ∑_{i=1}^{L} log σ(c_i ⋅ w)
log_P_positive_total = np.sum([np.log(p) for p in P_positive])

# Imprimimos los resultados
print("Embedding de la palabra objetivo w:")
print(w_embedding)
print("\nEmbeddings de las palabras de contexto c_i:")
for i, c_embedding in enumerate(context_embeddings):
    print(f"c_{i+1}:", c_embedding)

print("\nProductos punto c_i ⋅ w:")
for i, dot_product in enumerate(dot_products):
    print(f"c_{i+1} ⋅ w:", dot_product)

print("\nProbabilidades P(+|w, c_i) = σ(c_i ⋅ w):")
for i, p in enumerate(P_positive):
    print(f"P(+|w, c_{i+1}):", p)

print("\nProbabilidades P(-|w, c_i) = σ(-c_i ⋅ w):")
for i, p in enumerate(P_negative):
    print(f"P(-|w, c_{i+1}):", p)

print("\nProbabilidad total P(+|w, c_1:L) = producto de P(+|w, c_i):")
print("P(+|w, c_1:L):", P_positive_total)

print("\nLogaritmo de la probabilidad total log P(+|w, c_1:L):")
print("log P(+|w, c_1:L):", log_P_positive_total)


### El algoritmo de aprendizaje de **Skip-gram**

El algoritmo de **skip-gram** para aprender *embeddings* comienza con un corpus de texto y un vocabulario de tamaño $N$. Inicialmente, asigna vectores de *embedding* aleatorios a cada una de las $N$ palabras. Luego, ajusta iterativamente estos *embeddings* para que las palabras objetivo sean más similares a sus palabras de contexto reales y menos similares a palabras negativas (no relacionadas).

#### Proceso de entrenamiento

1. **Ejemplos de entrenamiento:**
   - **Positivos (+):** Pares $(w, c_{\text{pos}})$ donde $w$ es la palabra objetivo y $c_{\text{pos}}$ son palabras de contexto reales dentro de una ventana de contexto $L = \pm2$.
   - **Negativos (-):** Para cada ejemplo positivo, se generan $k$ pares negativos $(w, c_{\text{neg}})$ usando palabras de ruido seleccionadas aleatoriamente del vocabulario, excluyendo $w$. La probabilidad de selección de una palabra de ruido está ponderada por $p_{\alpha}(w)$ con $\alpha = 0.75$, lo que incrementa ligeramente la probabilidad de palabras raras.

2. **Función de pérdida:**
   - Se define una función de pérdida $L$ que busca maximizar la similitud entre $w$ y $c_{\text{pos}}$, mientras minimiza la similitud entre $w$ y cada $c_{\text{neg}}$.
   - La pérdida se expresa como:
     $$
     L = - \left[ \log \sigma(\mathbf{c}_{\text{pos}} \cdot \mathbf{w}) + \sum_{i=1}^{k} \log \sigma(-\mathbf{c}_{\text{neg}_i} \cdot \mathbf{w}) \right]
     $$
   - Aquí, $\sigma(x)$ es la función sigmoide que convierte el producto punto en una probabilidad.

3. **Actualización de *embeddings*:**
   - Utilizando descenso de gradiente estocástico, los vectores de *embedding* se actualizan para minimizar la función de pérdida.
   - Las actualizaciones ajustan tanto los *embeddings* de las palabras objetivo como los de las palabras de contexto y negativas.

4. **Representación final:**
   - Cada palabra tiene dos *embeddings*: uno objetivo ($\mathbf{w}_i$) y uno de contexto ($\mathbf{c}_i$).
   - Comúnmente, se suman estos vectores para obtener la representación final de la palabra: $\mathbf{w}_i + \mathbf{c}_i$. Alternativamente, se puede usar solo $\mathbf{w}_i$.

#### Consideraciones adicionales

- **Tamaño de la ventana de contexto ($L$):** Influye en la calidad de los *embeddings* y se suele ajustar experimentalmente.
- **Eficiencia:** **Skip-gram** con *negative sampling* es eficiente y permite entrenar *embeddings* densos y efectivos que capturan relaciones semánticas entre palabras.

En resumen, el algoritmo de **skip-gram** optimiza los *embeddings* de palabras mediante la maximización de la similitud entre palabras objetivo y sus contextos reales, y la minimización con palabras de ruido, utilizando un enfoque de auto-supervisión y descenso de gradiente para aprender representaciones vectoriales densas y útiles para tareas de procesamiento del lenguaje natural.

In [ ]:
import numpy as np

# Definimos la función sigmoide σ(x) = 1 / (1 + exp(-x))
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Función de pérdida L para un ejemplo de entrenamiento
def loss_function(w, c_pos, c_negs):
    # Producto punto c_pos ⋅ w
    dot_product_pos = np.dot(c_pos, w)
    # Calculamos σ(c_pos ⋅ w)
    sigma_pos = sigmoid(dot_product_pos)
    # Pérdida para el ejemplo positivo
    loss_pos = -np.log(sigma_pos + 1e-7)  # Añadimos un pequeño valor para evitar log(0)
    
    # Pérdida para los ejemplos negativos
    loss_negs = 0
    for c_neg in c_negs:
        dot_product_neg = np.dot(c_neg, w)
        sigma_neg = sigmoid(-dot_product_neg)
        loss_neg = -np.log(sigma_neg + 1e-7)
        loss_negs += loss_neg
    
    # Pérdida total
    total_loss = loss_pos + loss_negs
    return total_loss

# Funciones para calcular los gradientes
def gradients(w, c_pos, c_negs):
    # Gradiente respecto a c_pos
    dot_product_pos = np.dot(c_pos, w)
    sigma_pos = sigmoid(dot_product_pos)
    grad_c_pos = (sigma_pos - 1) * w  # ∂L/∂c_pos
    
    # Gradiente respecto a c_negs
    grad_c_negs = []
    for c_neg in c_negs:
        dot_product_neg = np.dot(c_neg, w)
        sigma_neg = sigmoid(np.dot(c_neg, w))
        grad_c_neg = sigma_neg * w  # ∂L/∂c_neg
        grad_c_negs.append(grad_c_neg)
    
    # Gradiente respecto a w
    grad_w = (sigma_pos - 1) * c_pos
    for i, c_neg in enumerate(c_negs):
        dot_product_neg = np.dot(c_neg, w)
        sigma_neg = sigmoid(dot_product_neg)
        grad_w += sigma_neg * c_neg  # Sumatoria para cada c_neg_i
        
    return grad_w, grad_c_pos, grad_c_negs

# Parámetros iniciales
np.random.seed(42)
embedding_size = 5  # Dimensión de los embeddings
learning_rate = 0.01  # Tasa de aprendizaje
k = 2  # Número de muestras negativas por ejemplo positivo

# Palabra objetivo y palabras de contexto positivas
w_word = 'apricot'
context_positive_words = ['tablespoon', 'of', 'jam', 'a']

# Vocabulario simulado (incluyendo palabras de ruido)
vocab = ['apricot', 'tablespoon', 'of', 'jam', 'a', 'aardvark', 'my', 'where', 'coaxial', 'seven', 'forever', 'dear', 'if']

# Inicializamos los embeddings para cada palabra en el vocabulario
embeddings = {word: np.random.randn(embedding_size) for word in vocab}

# Función para muestrear palabras negativas según Pα(w)
def sample_negative_words(vocab, positive_words, w_word, alpha=0.75, k=2):
    # Contamos la frecuencia simulada de cada palabra (para este ejemplo, asignamos valores arbitrarios)
    word_counts = {'apricot': 5, 'tablespoon': 10, 'of': 50, 'jam': 15, 'a': 40,
                   'aardvark': 1, 'my': 30, 'where': 20, 'coaxial': 2, 'seven': 7,
                   'forever': 3, 'dear': 4, 'if': 25}
    
    total_count = sum(count ** alpha for count in word_counts.values())
    word_probs = {word: (count ** alpha) / total_count for word, count in word_counts.items()}
    
    # Excluimos las palabras positivas y la palabra objetivo
    negative_vocab = [word for word in vocab if word not in positive_words and word != w_word]
    negative_probs = [word_probs[word] for word in negative_vocab]
    
    # Normalizamos las probabilidades para que sumen 1
    prob_sum = sum(negative_probs)
    normalized_probs = [prob / prob_sum for prob in negative_probs]
    
    # Muestreamos palabras negativas
    negative_words = np.random.choice(negative_vocab, size=k, p=normalized_probs)
    return negative_words

# Iteramos sobre los ejemplos de entrenamiento
for epoch in range(1):  # Para simplicidad, solo una época
    print(f"\nEpoca {epoch+1}")
    total_loss = 0
    for c_pos_word in context_positive_words:
        w = embeddings[w_word]
        c_pos = embeddings[c_pos_word]
        
        # Muestreamos k palabras negativas
        negative_words = sample_negative_words(vocab, context_positive_words, w_word, k=k)
        c_negs = [embeddings[neg_word] for neg_word in negative_words]
        
        # Calculamos la pérdida
        loss = loss_function(w, c_pos, c_negs)
        total_loss += loss
        
        # Calculamos los gradientes
        grad_w, grad_c_pos, grad_c_negs = gradients(w, c_pos, c_negs)
        
        # Actualizamos los embeddings
        embeddings[w_word] -= learning_rate * grad_w
        embeddings[c_pos_word] -= learning_rate * grad_c_pos
        for i, neg_word in enumerate(negative_words):
            embeddings[neg_word] -= learning_rate * grad_c_negs[i]
        
        # Imprimimos información del entrenamiento
        print(f"\nPalabra objetivo: '{w_word}', Palabra de contexto positiva: '{c_pos_word}'")
        print(f"Palabras negativas: {negative_words}")
        print(f"Pérdida: {loss}")
        print(f"Gradiente w: {grad_w}")
        print(f"Gradiente c_pos ('{c_pos_word}'): {grad_c_pos}")
        for i, neg_word in enumerate(negative_words):
            print(f"Gradiente c_neg_{i+1} ('{neg_word}'): {grad_c_negs[i]}")
    
    print(f"\nPérdida total en la época {epoch+1}: {total_loss}")


### Otros tipos de *embeddings* estáticos

Existen muchos tipos de *embeddings* estáticos. Una extensión de **word2vec**, **fasttext** , aborda un problema con **word2vec** tal como lo hemos presentado hasta ahora: no tiene una buena manera de manejar palabras desconocidas, es decir, palabras que aparecen en un corpus de prueba pero no se vieron en el corpus de entrenamiento. Un problema relacionado es la dispersión de palabras, como en lenguas con morfología rica, donde algunas de las muchas formas para cada sustantivo y verbo pueden aparecer solo raramente. 

**Fasttext** aborda estos problemas utilizando modelos de subpalabras, representando cada palabra como sí misma más un conjunto de n-gramas constituyentes, con símbolos especiales de frontera `<` y `>` añadidos a cada palabra. Por ejemplo, con $n = 3$, la palabra *where* se representaría por la secuencia `<where>` más los n-gramas de caracteres:

$$
\text{<wh, whe, her, ere, re>}
$$

Luego se aprende un *embedding* de **skipgram** para cada n-grama constituyente, y la palabra *where* se representa por la suma de todos los *embeddings* de sus n-gramas constituyentes. Las palabras desconocidas pueden representarse solo por la suma de los n-gramas constituyentes. Una biblioteca de código abierto de **fasttext**, que incluye *embeddings* preentrenados para 157 idiomas, está disponible en [https://fasttext.cc](https://fasttext.cc).

Otro modelo de *embedding* estático muy utilizado es **GloVe**, abreviatura de *Global Vectors*, ya que el modelo se basa en capturar estadísticas globales del corpus. **GloVe** se basa en proporciones de probabilidades de la matriz de coocurrencia palabra-palabra, combinando las intuiciones de los modelos basados en conteo como **PPMI** y las estructuras lineales utilizadas por métodos como **word2vec**.



In [ ]:
#pip install fasttext


In [ ]:
"""
import os
import urllib.request
import gzip
import shutil
from pathlib import Path

# Definir rutas
base_dir = Path.cwd()  # Directorio actual del notebook
fasttext_dir = base_dir / 'models' / 'fasttext'
fasttext_dir.mkdir(parents=True, exist_ok=True)  # Crear directorios si no existen

# URL del modelo preentrenado para español
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz'

# Rutas de los archivos
gz_path = fasttext_dir / 'cc.es.300.bin.gz'
bin_path = fasttext_dir / 'cc.es.300.bin'

# Función para descargar el archivo
def descargar_modelo(url, destino):
    if not destino.exists():
        print(f"Descargando {url}...")
        urllib.request.urlretrieve(url, destino)
        print("Descarga completada.")
    else:
        print(f"El archivo {destino} ya existe. Se omitirá la descarga.")

# Función para descomprimir el archivo .gz
def descomprimir_gz(origen, destino):
    if not destino.exists():
        print(f"Descomprimiendo {origen}...")
        with gzip.open(origen, 'rb') as f_in:
            with open(destino, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print("Descompresión completada.")
    else:
        print(f"El archivo descomprimido {destino} ya existe. Se omitirá la descompresión.")

# Descargar el modelo preentrenado
descargar_modelo(url, gz_path)

# Descomprimir el modelo
descomprimir_gz(gz_path, bin_path)

# Verificar la existencia del archivo descomprimido
if bin_path.exists():
    print(f"El modelo fastText está listo en: {bin_path}")
else:
    print(f"Hubo un problema al descomprimir el modelo {gz_path}.")
"""


In [ ]:
"""
import fasttext

# Ruta al modelo preentrenado descargado y descomprimido
modelo_fasttext_path = 'models/fasttext/cc.es.300.bin'

# Cargar el modelo preentrenado
modelo = fasttext.load_model(modelo_fasttext_path)

# Obtener el vector para una palabra conocida
vector_conocida = modelo.get_word_vector('donde')
print(f"Vector para 'donde':\n{vector_conocida}\n")

# Obtener el vector para una palabra desconocida
vector_desconocida = modelo.get_word_vector('dondeque')
print(f"Vector para 'dondeque' (desconocida):\n{vector_desconocida}\n")
"""

### Embeddings densos como word2vec y embeddings dispersos como PPMI


En el procesamiento del lenguaje natural (NLP), una de las tareas fundamentales es representar palabras de manera que las máquinas puedan entender y procesar el lenguaje humano. Las palabras, en su forma textual, no son directamente procesables por algoritmos matemáticos o modelos estadísticos. Por lo tanto, es esencial convertir estas palabras en representaciones numéricas, conocidas como *embeddings*, que capturan relaciones semánticas y sintácticas entre ellas.

Existen dos enfoques principales para crear estos embeddings: los *embeddings densos*, como word2vec, y los *embeddings dispersos*, como los basados en PPMI (Positive Pointwise Mutual Information). Este informe explorará en detalle ambos métodos, incluyendo las ecuaciones fundamentales, la implementación de matrices de co-ocurrencia y el cálculo de los valores de PPMI.


**Embeddings Densos: word2vec**

Los embeddings densos son vectores de baja dimensionalidad donde cada dimensión es un número real que captura características latentes de las palabras. Word2vec, desarrollado por Mikolov et al. en 2013, es uno de los métodos más populares para generar embeddings densos.

*Arquitecturas de word2vec*

Word2vec tiene dos arquitecturas principales:

1. **Continuous Bag-of-Words (CBOW):** Predice la palabra objetivo basada en su contexto.

2. **Skip-Gram:** Predice las palabras de contexto basadas en la palabra objetivo.

Ambas arquitecturas utilizan una red neuronal simple para aprender los embeddings.

*Modelo Skip-Gram*

El objetivo del modelo Skip-Gram es maximizar la probabilidad de que, dada una palabra central $w_t$, se puedan predecir las palabras de contexto $w_{t+j}$ dentro de una ventana de contexto definida.

La función de probabilidad para el modelo Skip-Gram es:

$$
\max \prod_{t=1}^{T} \prod_{-c \leq j \leq c, j \neq 0} P(w_{t+j} | w_t)
$$

Donde:

- $T$ es el número total de palabras en el corpus.
- $c$ es el tamaño de la ventana de contexto.

*Función de Probabilidad Condicional*

La probabilidad condicional $P(w_{O} | w_{I})$ se modela utilizando la función softmax:


$$
P(w_{O} | w_{I}) = \frac{\exp(\mathbf{v}_{w_{O}}^\top \mathbf{v}_{w_{I}})}{\sum_{w=1}^{|V|} \exp(\mathbf{v}_{w}^\top \mathbf{v}_{w_{I}})}
$$


Donde:

- $\mathbf{v}_{w_{I}}$ es el vector de entrada de la palabra objetivo.
- $\mathbf{v}'_{w_{O}}$es el vector de salida de la palabra de contexto.
- $|V|$ es el tamaño del vocabulario.

*Optimización*

El cálculo directo de la función softmax es computacionalmente costoso para grandes vocabularios. Para abordar esto, se utilizan métodos como *negative sampling* y *hierarchical softmax*.

- **Negative sampling:** En lugar de actualizar todo el vocabulario, se actualiza un subconjunto de palabras negativas.
  
- **Hierarchical softmax:** Utiliza una estructura de árbol para reducir la complejidad computacional.

*Ecuaciones de actualización*

Durante el entrenamiento, los vectores de las palabras se actualizan utilizando gradientes derivados de la función de pérdida. Por ejemplo, utilizando *Negative Sampling*, la función de pérdida para una pareja $(w_{I}, w_{O})$ es:



$$
L = -\log \sigma(\mathbf{v}_{w_{O}}^\top \mathbf{v}_{w_{I}}) - \sum_{i=1}^{k} \log \sigma(-\mathbf{v}_{w_{i}}^\top \mathbf{v}_{w_{I}})
$$

Esta es la forma corregida de la ecuación, adecuada para el contexto de *skip-gram* con *negative sampling*.

Donde:

- $\sigma(x)$ es la función sigmoide.
- $k$ es el número de muestras negativas.
- $w_{i}$ son las palabras negativas.

**Embeddings dispersos: PPMI**

Los embeddings dispersos representan palabras en vectores de alta dimensionalidad donde la mayoría de las entradas son cero. Una técnica común para crear estos embeddings es utilizar la Información Mutua Puntual Positiva (PPMI).

*Información Mutua Puntual (PMI)*

La PMI mide la asociación entre dos eventos, en este caso, dos palabras. Se define como:

$$
\text{PMI}(w, c) = \log \left( \frac{P(w, c)}{P(w)P(c)} \right)
$$

Donde:

- $P(w, c)$ es la probabilidad conjunta de las palabras $w$ y $c$.
- $P(w)$ y $P(c)$ son las probabilidades marginales.

Un valor de PMI alto indica una asociación fuerte entre $w$ y $c$.

*PPMI (Positive PMI)*

La PMI puede producir valores negativos cuando $P(w, c) < P(w)P(c)$. Para enfocarse en asociaciones positivas, se utiliza PPMI:

$$
\text{PPMI}(w, c) = \max( \text{PMI}(w, c), 0 )
$$


**Implementación de la matriz de co-ocurrencia**

La matriz de co-ocurrencia es esencial para calcular los valores de PPMI. Cada entrada en la matriz representa la frecuencia con la que dos palabras aparecen juntas en un contexto definido.

*Pasos para Construir la Matriz*

1. **Definir el Vocabulario $V$:** Extraer todas las palabras únicas del corpus.

2. **Inicializar la Matriz $M$:** Crear una matriz de $|V| \times |V|$ inicializada en cero.

3. **Recorrer el Corpus:**

   - Para cada palabra $w_i$, identificar su contexto $C(w_i)$ dentro de una ventana de tamaño $k$.
   - Incrementar $M_{w_i, w_j}$ en uno para cada $w_j \in C(w_i)$.

*Ejemplo:*

Si la oración es "el perro ladra fuerte", y la ventana de contexto es de tamaño 2, las co-ocurrencias serían:

- "el" con "perro" y "ladra".
- "perro" con "el", "ladra" y "fuerte".
- "ladra" con "el", "perro", "fuerte".
- "fuerte" con "perro" y "ladra".

*Calculando probabilidades*

- **Frecuencia total de co-ocurrencias $N$:**

$$
N = \sum_{w \in V} \sum_{c \in V} M_{w,c}
$$

- **Probabilidad conjunta $P(w, c)$:**

$$
P(w, c) = \frac{M_{w,c}}{N}
$$

- **Probabilidades marginales:**

$$
P(w) = \frac{\sum_{c \in V} M_{w,c}}{N}
$$
$$
P(c) = \frac{\sum_{w \in V} M_{w,c}}{N}
$$


**Cálculo de valores de PPMI**

Con las probabilidades calculadas, se procede a calcular los valores de PMI y luego aplicar PPMI.

*Paso 1: Calcular PMI para cada par $(w, c)$*

$$
\text{PMI}(w, c) = \log_2 \left( \frac{P(w, c)}{P(w)P(c)} \right)
$$

*Paso 2: Aplicar PPMI*

$$
\text{PPMI}(w, c) = \max( \text{PMI}(w, c), 0 )
$$

*Interpretación de PPMI*

Un valor de PPMI alto indica que las palabras $w$ y $c$ co-ocurren más frecuentemente de lo esperado si fueran independientes.


**Comparación entre embeddings densos y dispersos**

*Embeddings densos (word2vec)*

- **Ventajas:**
  - Capturan relaciones semánticas y sintácticas complejas.
  - Dimensionalidad reducida (típicamente entre 100 y 300 dimensiones).
  - Eficientes en términos de almacenamiento y cómputo.
  
- **Desventajas:**
  - Menos interpretables; las dimensiones no tienen un significado específico.
  - Requieren grandes cantidades de datos para entrenar eficazmente.
  - Dependientes del corpus utilizado para el entrenamiento.

*Embeddings dispersos (PPMI)*

- **Ventajas:**
  - Más interpretables; cada dimensión corresponde a una palabra del vocabulario.
  - Basados en estadísticas directas del corpus.
  - No requieren entrenamiento complejo.

- **Desventajas:**
  - Alta dimensionalidad (igual al tamaño del vocabulario), lo que aumenta los requerimientos de almacenamiento.
  - Pueden ser menos efectivos para capturar relaciones semánticas complejas.
  - Menos eficientes computacionalmente para tareas de aprendizaje automático.


**Consideraciones de implementación**

*Eficiencia y escalabilidad*

- **Almacenamiento disperso:** Utilizar estructuras de datos dispersas (como matrices dispersas) para almacenar la matriz de co-ocurrencia y los embeddings PPMI, lo que reduce significativamente el uso de memoria.

- **Reducción de dimensionalidad:** Aplicar técnicas como la Descomposición en Valores Singulares (SVD) para reducir la dimensionalidad de los embeddings dispersos, manteniendo la mayor parte de la información relevante.

*Optimización del cálculo*

- **Filtrado de palabras frecuentes/infrecuentes:** Ignorar palabras demasiado frecuentes (como "el", "la", "de") y palabras muy infrecuentes para reducir el ruido y el tamaño de la matriz.

- **Limitación del contexto:** Ajustar el tamaño de la ventana de contexto para equilibrar la cantidad de información capturada y la complejidad computacional.


**Ejemplo práctico: Cálculo de PPMI**

Supongamos un vocabulario reducido y una matriz de co-ocurrencia simplificada.

*Vocabulario:* $V = \{ \text{perro}, \text{gato}, \text{animal} \}$

*Matriz de Co-ocurrencia $M$:*

|       | perro | gato | animal |
|-------|-------|------|--------|
| perro |   0   |  3   |   2    |
| gato  |   3   |  0   |   1    |
| animal|   2   |  1   |   0    |

*Paso 1: Calcular el total de co-ocurrencias $N$*

$$
N = \sum_{w \in V} \sum_{c \in V} M_{w,c} = (0+3+2)+(3+0+1)+(2+1+0) = 12
$$

*Paso 2: Calcular probabilidades marginales*

- $P(\text{perro}) = \frac{0+3+2}{12} = \frac{5}{12}$
- $P(\text{gato}) = \frac{3+0+1}{12} = \frac{4}{12}$
- $P(\text{animal}) = \frac{2+1+0}{12} = \frac{3}{12}$

*Paso 3: Calcular probabilidades conjuntas*

- $P(\text{perro}, \text{gato}) = \frac{M_{\text{perro}, \text{gato}}}{N} = \frac{3}{12} = \frac{1}{4}$
- $P(\text{perro}, \text{animal}) = \frac{2}{12} = \frac{1}{6}$

*Paso 4: Calcular PMI*

- Para $(\text{perro}, \text{gato})$:

$$
\text{PMI}(\text{perro}, \text{gato}) = \log_2 \left( \frac{P(\text{perro}, \text{gato})}{P(\text{perro})P(\text{gato})} \right) = \log_2 \left( \frac{\frac{1}{4}}{\frac{5}{12} \times \frac{4}{12}} \right) = \log_2 \left( \frac{\frac{1}{4}}{\frac{20}{144}} \right) = \log_2 \left( \frac{36}{20} \right) \approx \log_2(1.8) \approx 0.85
$$

- Para $(\text{perro}, \text{animal})$:

$$
\text{PMI}(\text{perro}, \text{animal}) = \log_2 \left( \frac{\frac{1}{6}}{\frac{5}{12} \times \frac{3}{12}} \right) = \log_2 \left( \frac{\frac{1}{6}}{\frac{15}{144}} \right) = \log_2 \left( \frac{24}{15} \right) \approx \log_2(1.6) \approx 0.68
$$

*Paso 5: Calcular PPMI*

Como ambos valores de PMI son positivos, los valores de PPMI serán los mismos.


In [ ]:
"""
# Importar las bibliotecas necesarias
import numpy as np
from collections import Counter
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

# Definir el corpus
corpus = [
    "el perro ladra fuerte",
    "el gato maúlla suavemente",
    "el perro y el gato son amigos",
    "el animal ladra y maúlla"
]

# Preprocesamiento básico
def preprocess_text(text):
    return text.lower().split()

# Procesar el corpus
processed_corpus = [preprocess_text(doc) for doc in corpus]

# Construcción del vocabulario
word_counts = Counter()
for doc in processed_corpus:
    word_counts.update(doc)

# Crear un índice para cada palabra
vocab = {word: idx for idx, word in enumerate(word_counts.keys())}
index_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

print("Vocabulario:", vocab)

# Construcción de la matriz de co-ocurrencia
def build_cooccurrence_matrix(corpus, vocab, window_size=2):
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size))
    
    for doc in corpus:
        doc_length = len(doc)
        for idx, word in enumerate(doc):
            word_id = vocab[word]
            context_start = max(0, idx - window_size)
            context_end = min(doc_length, idx + window_size + 1)
            
            for context_idx in range(context_start, context_end):
                if context_idx != idx:
                    context_word = doc[context_idx]
                    context_word_id = vocab[context_word]
                    cooccurrence_matrix[word_id][context_word_id] += 1
    return cooccurrence_matrix

# Construir la matriz de co-ocurrencia
cooccurrence_matrix = build_cooccurrence_matrix(processed_corpus, vocab)

print("\nMatriz de Co-ocurrencia:")
print(cooccurrence_matrix)

# Cálculo de probabilidades marginales y conjuntas
N = np.sum(cooccurrence_matrix)
word_probabilities = np.sum(cooccurrence_matrix, axis=1) / N
context_probabilities = np.sum(cooccurrence_matrix, axis=0) / N
joint_probabilities = cooccurrence_matrix / N

# Cálculo de PMI y PPMI
with np.errstate(divide='ignore'):
    pmi_matrix = np.log2(joint_probabilities / (word_probabilities[:, None] * context_probabilities[None, :]))
    pmi_matrix[np.isinf(pmi_matrix)] = 0.0  # Reemplazar infinito por cero
    pmi_matrix[np.isnan(pmi_matrix)] = 0.0  # Reemplazar NaN por cero

ppmi_matrix = np.maximum(pmi_matrix, 0)

print("\nMatriz PPMI:")
print(ppmi_matrix)

# Función para imprimir la matriz con etiquetas
def print_matrix(matrix, vocab):
    print(" ", end="\t")
    for idx in range(vocab_size):
        print(index_to_word[idx], end="\t")
    print()
    for i in range(vocab_size):
        print(index_to_word[i], end="\t")
        for j in range(vocab_size):
            print(f"{matrix[i][j]:.2f}", end="\t")
        print()

print("\nMatriz de Co-ocurrencia (con etiquetas):")
print_matrix(cooccurrence_matrix, vocab)

print("\nMatriz PPMI (con etiquetas):")
print_matrix(ppmi_matrix, vocab)

# Reducción de dimensionalidad con SVD
n_components = 2  # Reducimos a 2 dimensiones para visualización
svd = TruncatedSVD(n_components=n_components)
ppmi_reduced = svd.fit_transform(ppmi_matrix)

print("\nEmbeddings PPMI Reducidos:")
for i in range(len(ppmi_reduced)):
    print(f"{index_to_word[i]}: {ppmi_reduced[i]}")

# Visualización de los embeddings PPMI reducidos
plt.figure(figsize=(8, 6))
for i in range(len(ppmi_reduced)):
    plt.scatter(ppmi_reduced[i, 0], ppmi_reduced[i, 1])
    plt.text(ppmi_reduced[i, 0]+0.01, ppmi_reduced[i, 1]+0.01, index_to_word[i])

plt.title("Visualización de Embeddings PPMI Reducidos")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

# Implementación de word2vec con gensim
w2v_model = Word2Vec(sentences=processed_corpus, vector_size=2, window=2, min_count=1, workers=1, sg=1, epochs=100)

# Obtener los vectores de word2vec
word_vectors = w2v_model.wv
w2v_embeddings = np.array([word_vectors[word] for word in vocab.keys()])

print("\nEmbeddings word2vec:")
for i, word in enumerate(vocab.keys()):
    print(f"{word}: {w2v_embeddings[i]}")

# Visualización de los embeddings word2vec
plt.figure(figsize=(8, 6))
for i, word in enumerate(vocab.keys()):
    vector = w2v_embeddings[i]
    plt.scatter(vector[0], vector[1])
    plt.text(vector[0]+0.01, vector[1]+0.01, word)

plt.title("Visualización de Embeddings word2vec")
plt.xlabel("Dimensión 1")
plt.ylabel("Dimensión 2")
plt.grid(True)
plt.show()
"""

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
from collections import Counter
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

# Definir el corpus
corpus = [
    "el perro ladra fuerte",
    "el gato maúlla suavemente",
    "el perro y el gato son amigos",
    "el animal ladra y maúlla"
]

# Preprocesamiento básico
def preprocess_text(text):
    return text.lower().split()

# Procesar el corpus
processed_corpus = [preprocess_text(doc) for doc in corpus]

# Construcción del vocabulario
word_counts = Counter()
for doc in processed_corpus:
    word_counts.update(doc)

# Crear un índice para cada palabra
vocab = {word: idx for idx, word in enumerate(word_counts.keys())}
index_to_word = {idx: word for word, idx in vocab.items()}
vocab_size = len(vocab)

print("Vocabulario:", vocab)

# Construcción de la matriz de co-ocurrencia
def build_cooccurrence_matrix(corpus, vocab, window_size=2):
    cooccurrence_matrix = np.zeros((vocab_size, vocab_size))
    
    for doc in corpus:
        doc_length = len(doc)
        for idx, word in enumerate(doc):
            word_id = vocab[word]
            context_start = max(0, idx - window_size)
            context_end = min(doc_length, idx + window_size + 1)
            
            for context_idx in range(context_start, context_end):
                if context_idx != idx:
                    context_word = doc[context_idx]
                    context_word_id = vocab[context_word]
                    cooccurrence_matrix[word_id][context_word_id] += 1
    return cooccurrence_matrix

# Construir la matriz de co-ocurrencia
cooccurrence_matrix = build_cooccurrence_matrix(processed_corpus, vocab)

print("\nMatriz de Co-ocurrencia:")
print(cooccurrence_matrix)

# Cálculo de probabilidades marginales y conjuntas
N = np.sum(cooccurrence_matrix)
word_probabilities = np.sum(cooccurrence_matrix, axis=1) / N
context_probabilities = np.sum(cooccurrence_matrix, axis=0) / N
joint_probabilities = cooccurrence_matrix / N

# Cálculo de PMI y PPMI
with np.errstate(divide='ignore'):
    pmi_matrix = np.log2(joint_probabilities / (word_probabilities[:, None] * context_probabilities[None, :]))
    pmi_matrix[np.isinf(pmi_matrix)] = 0.0  # Reemplazar infinito por cero
    pmi_matrix[np.isnan(pmi_matrix)] = 0.0  # Reemplazar NaN por cero

ppmi_matrix = np.maximum(pmi_matrix, 0)

print("\nMatriz PPMI:")
print(ppmi_matrix)

# Función para imprimir la matriz con etiquetas
def print_matrix(matrix, vocab):
    print(" ", end="\t")
    for idx in range(vocab_size):
        print(index_to_word[idx], end="\t")
    print()
    for i in range(vocab_size):
        print(index_to_word[i], end="\t")
        for j in range(vocab_size):
            print(f"{matrix[i][j]:.2f}", end="\t")
        print()

print("\nMatriz de Co-ocurrencia (con etiquetas):")
print_matrix(cooccurrence_matrix, vocab)

print("\nMatriz PPMI (con etiquetas):")
print_matrix(ppmi_matrix, vocab)

# Reducción de dimensionalidad con SVD
n_components = 2  # Reducimos a 2 dimensiones para visualización
svd = TruncatedSVD(n_components=n_components)
ppmi_reduced = svd.fit_transform(ppmi_matrix)

print("\nEmbeddings PPMI Reducidos:")
for i in range(len(ppmi_reduced)):
    print(f"{index_to_word[i]}: {ppmi_reduced[i]}")

# Visualización de los embeddings PPMI reducidos
plt.figure(figsize=(8, 6))
for i in range(len(ppmi_reduced)):
    plt.scatter(ppmi_reduced[i, 0], ppmi_reduced[i, 1])
    plt.text(ppmi_reduced[i, 0]+0.01, ppmi_reduced[i, 1]+0.01, index_to_word[i])

plt.title("Visualización de Embeddings PPMI Reducidos")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

# ---- Implementación de Word2Vec sin usar gensim ----

# Parámetros del modelo
embedding_dim = 2  # Dimensionalidad de los embeddings
learning_rate = 0.01
epochs = 1000
window_size = 2

# Generar pares (objetivo, contexto)
def generate_training_data(corpus, window_size):
    training_data = []
    for sentence in corpus:
        sentence_length = len(sentence)
        for idx, target_word in enumerate(sentence):
            context_start = max(0, idx - window_size)
            context_end = min(sentence_length, idx + window_size + 1)
            for context_idx in range(context_start, context_end):
                if context_idx != idx:
                    context_word = sentence[context_idx]
                    training_data.append((vocab[target_word], vocab[context_word]))
    return training_data

training_data = generate_training_data(processed_corpus, window_size)
print("\nPares de entrenamiento (objetivo, contexto):")
print(training_data)

# Inicialización de matrices de pesos
np.random.seed(0)  # Para reproducibilidad
W_in = np.random.uniform(-0.8, 0.8, (vocab_size, embedding_dim))
W_out = np.random.uniform(-0.8, 0.8, (vocab_size, embedding_dim))

# Funciones auxiliares
def softmax(x):
    e_x = np.exp(x - np.max(x))  # Estabilidad numérica
    return e_x / e_x.sum(axis=0)

def train_skipgram(training_data, W_in, W_out, epochs, learning_rate):
    for epoch in range(epochs):
        loss = 0
        for target, context in training_data:
            # Forward pass
            z = np.dot(W_out, W_in[target])
            y_pred = softmax(z)
            
            # Cálculo de la pérdida (cross-entropy)
            loss -= np.log(y_pred[context] + 1e-7)  # Evitar log(0)
            
            # Backward pass
            y_pred[context] -= 1  # Derivada de la pérdida respecto a z
            
            # Gradientes
            dW_out = np.outer(y_pred, W_in[target])
            dW_in = np.dot(W_out.T, y_pred)
            
            # Actualización de pesos
            W_out -= learning_rate * dW_out
            W_in[target] -= learning_rate * dW_in
        if (epoch + 1) % 100 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs}, Pérdida: {loss:.4f}")
    return W_in, W_out

# Entrenar el modelo Skip-Gram
W_in, W_out = train_skipgram(training_data, W_in, W_out, epochs, learning_rate)

# Extraer embeddings de palabras
word_embeddings = W_in

print("\nEmbeddings Word2Vec personalizados:")
for word, idx in vocab.items():
    print(f"{word}: {word_embeddings[idx]}")

# Visualización de los embeddings Word2Vec personalizados
plt.figure(figsize=(8, 6))
for word, idx in vocab.items():
    vector = word_embeddings[idx]
    plt.scatter(vector[0], vector[1])
    plt.text(vector[0]+0.01, vector[1]+0.01, word)

plt.title("Visualización de Embeddings Word2Vec Personalizados")
plt.xlabel("Dimensión 1")
plt.ylabel("Dimensión 2")
plt.grid(True)
plt.show()


### Ejercicios

**Ejercicio 1: Comprensión del preprocesamiento del corpus**

**a.** Explica por qué es importante el preprocesamiento del texto antes de construir el vocabulario y la matriz de co-ocurrencia. ¿Qué técnicas de preprocesamiento adicionales podrían implementarse para mejorar la calidad de los embeddings?

**b.** Dado el corpus proporcionado, identifica manualmente el vocabulario y asigne un índice a cada palabra. ¿Coinciden tus resultados con los obtenidos en el código?


**Ejercicio 2: Construcción manual de la matriz de co-ocurrencia**

**a.** Utilizando una ventana de contexto de tamaño 2, construya manualmente la matriz de co-ocurrencia para el corpus dado. Asegúrate de considerar todas las oraciones y de contar correctamente las co-ocurrencias.

**b.** Compara tu matriz con la generada en el código. Identifica y explica cualquier discrepancia.


**Ejercicio 3: Cálculo de probabilidades y PMI**

**a.** Calcula el total de co-ocurrencias ($N$) basado en la matriz de co-ocurrencia que construyó en el ejercicio anterior.

**b.** Calcula las probabilidades marginales $P(w)$ y $P(c)$ para cada palabra del vocabulario.

**c.** Calcula las probabilidades conjuntas $P(w, c)$ para cada par de palabras que co-ocurren.

**d.** Utilizando las probabilidades calculadas, determina los valores de PMI para al menos cinco pares de palabras con co-ocurrencias significativas.


**Ejercicio 4: Aplicación de PPMI**

**a.** A partir de los valores de PMI obtenidos, aplique la función PPMI para obtener los valores positivos. ¿Qué efecto tiene esta transformación en los valores negativos de PMI?

**b.** Analiza los valores de PPMI obtenidos. ¿Cuáles son las palabras que presentan la mayor asociación? Explica por qué, considerando el contexto del corpus.


**Ejercicio 5: Interpretación de la matriz PPMI**

**a.** Examina la matriz PPMI resultante y explique qué representa cada dimensión en este espacio vectorial.

**b.** Discute las ventajas y desventajas de utilizar embeddings dispersos de alta dimensionalidad en aplicaciones de PLN.


**Ejercicio 6: Reducción de dimensionalidad con SVD**

**a.** Explica el propósito de aplicar la Descomposición en Valores Singulares (SVD) a la matriz PPMI.

**b.** ¿Qué significan los componentes resultantes después de la reducción de dimensionalidad? ¿Se mantiene la interpretabilidad de las dimensiones?

**c.** Discute cómo la elección del número de componentes afecta la calidad y utilidad de los embeddings reducidos.


**Ejercicio 7: Visualización y análisis de embeddings PPMI reducidos**

**a.** Basándote en la visualización de los embeddings PPMI reducidos a dos dimensiones, identifica grupos de palabras que estén cercanas en el espacio vectorial. ¿Qué relaciones semánticas o sintácticas comparten estas palabras?

**b.** Propone una interpretación para la posición relativa de las palabras "perro", "gato", "animal", "ladra" y "maúlla" en el espacio reducido.


**Ejercicio 8: Implementación y comprensión de word2vec**

**a.** Explica brevemente cómo funciona el modelo skip-gram con negative sampling utilizado en word2vec.

**b.** Discute cómo los parámetros del modelo (por ejemplo, tamaño de ventana, dimensiones del vector, número de épocas) pueden influir en los embeddings resultantes.

**c.** Sin ejecutar código, describa los pasos necesarios para entrenar un modelo word2vec con el corpus dado.

**Ejercicio 9: Comparación entre embeddings PPMI y word2vec**

**a.** Compara las visualizaciones de los embeddings PPMI reducidos y los embeddings word2vec. ¿Qué similitudes y diferencias observa en la distribución de las palabras en ambos espacios?

**b.** Analiza cómo cada método captura las relaciones entre palabras. Proporciona ejemplos específicos del corpus para respaldar su análisis.


**Ejercicio 10: Aplicación práctica de los embeddings**

**a.** Proponiendo una tarea sencilla de NLP (por ejemplo, clasificación de textos o detección de sinónimos), explica cómo utilizarías los embeddings PPMI y word2vec para resolverla.

**b.** Discute las posibles ventajas y limitaciones de cada tipo de embedding en el contexto de la tarea propuesta.

**Ejercicio 11: Impacto del tamaño del corpus**

**a.** Reflexiona sobre cómo el tamaño y la diversidad del corpus afectan la calidad de los embeddings generados tanto por PPMI como por word2vec.

**b.** Si tuvieras la oportunidad de ampliar el corpus, ¿qué tipo de datos adicionales incluiría y por qué?


**Ejercicio 12: Ajuste de parámetros y optimización**

**a.** Supongamos que experimentas con diferentes tamaños de ventana en la construcción de la matriz de co-ocurrencia. ¿Cómo esperas que esto afecte los valores de PPMI y, en consecuencia, los embeddings?

**b.** Analiza cómo el parámetro `min_count` en word2vec influye en el vocabulario y en los embeddings generados.


**Ejercicio 13: Exploración de palabras raras y rrecuentes**

**a.** Considera las palabras muy frecuentes y muy infrecuentes en el corpus. ¿Cómo afectan estas palabras a la matriz de co-ocurrencia y al cálculo de PPMI?

**b.** Discute estrategias para manejar palabras de alta frecuencia (como "el", "y") al construir embeddings. ¿Por qué podría ser útil filtrarlas o asignarles un peso diferente?


**Ejercicio 14: Interpretación de resultados y conclusiones**

**a.** Resume las principales diferencias entre los embeddings densos y dispersos en términos de representación y uso práctico.

**b.** Basándose en los ejercicios anteriores, ¿cuál método consideras más adecuado para aplicaciones específicas de PLN y por qué?

**Ejercicio 15: Investigación adicional**

**a.** Investiga otras técnicas de embeddings dispersos y densos no mencionadas en el texto (por ejemplo, GloVe, FastText). Compara sus enfoques y características principales con PPMI y word2vec.

**b.** Explora cómo incorporarías información contextual adicional (como etiquetado gramatical o entidades nombradas) en la generación de embeddings. ¿Qué modificaciones al código o al proceso serían necesarias?



In [ ]:
## Tus respuestas

### Visualización de *embeddings*

Visualizar *embeddings* es un objetivo importante para ayudar a comprender, aplicar y mejorar estos modelos de significado de palabras. Pero ¿cómo podemos visualizar un vector de (por ejemplo) 100 dimensiones?

La forma más sencilla de visualizar el significado de una palabra $w$ incrustada en un espacio es listar las palabras más similares a $w$ ordenando los vectores de todas las palabras en el vocabulario según su coseno con el vector de $w$. Por ejemplo, las 7 palabras más cercanas a *frog* usando un conjunto de *embeddings* calculados con el algoritmo **GloVe** son: *frogs*, *toad*, *litoria*, *leptodactylidae*, *rana*, *lizard*, y *eleutherodactylus*.

Otro método de visualización es usar un algoritmo de agrupamiento para mostrar una representación jerárquica de qué palabras son similares a otras en el espacio de *embeddings*.

Sin embargo, el método de visualización probablemente más común es proyectar las 100 dimensiones de una palabra a 2 dimensiones  utilizando un método de proyección llamado **t-SNE**.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import time  # Para medir el tiempo de computación

# Paso 1: Cargar los embeddings de GloVe
def cargar_glove(ruta_archivo):
    embeddings = {}
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        for linea in f:
            valores = linea.strip().split()
            palabra = valores[0]
            vector = np.array(valores[1:], dtype='float32')
            embeddings[palabra] = vector
    return embeddings

#https://github.com/rohanrao619/Twitter_Sentiment_Analysis/blob/master/glove.6B.100d.txt
ruta_glove = 'glove.6B.100d.txt'  # Asegúrate de tener este archivo en tu directorio de trabajo
embeddings = cargar_glove(ruta_glove)

# Paso 2: Seleccionar un subconjunto de palabras
# Puedes experimentar con diferentes conjuntos de palabras
palabras_interes = input("Ingresa las palabras que deseas visualizar, separadas por comas: ")
palabras_interes = [palabra.strip() for palabra in palabras_interes.split(',')]

# Filtrar las palabras que existen en los embeddings
palabras_existentes = [palabra for palabra in palabras_interes if palabra in embeddings]

# Verificar si hay palabras que no están en los embeddings
palabras_no_encontradas = set(palabras_interes) - set(palabras_existentes)
if palabras_no_encontradas:
    print(f"Advertencia: Las siguientes palabras no se encontraron en los embeddings y serán omitidas: {', '.join(palabras_no_encontradas)}")

# Obtener los vectores correspondientes
vectores = np.array([embeddings[palabra] for palabra in palabras_existentes])

# Paso 3: Ajustar parámetros de t-SNE
perplexity = int(input("Ingresa el valor de perplexity para t-SNE (recomendado entre 5 y 50): "))
n_iter = int(input("Ingresa el número de iteraciones para t-SNE (recomendado al menos 1000): "))

# Medir el tiempo de computación
inicio_tiempo = time.time()

# Aplicar t-SNE para reducir a 2 dimensiones
tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
vectores_tsne = tsne.fit_transform(vectores)

fin_tiempo = time.time()
tiempo_total = fin_tiempo - inicio_tiempo
print(f"Tiempo de computación de t-SNE: {tiempo_total:.2f} segundos")

# Paso 4: Graficar los resultados
plt.figure(figsize=(12, 8))
for i, palabra in enumerate(palabras_existentes):
    x, y = vectores_tsne[i, :]
    plt.scatter(x, y)
    plt.annotate(palabra, (x, y), textcoords="offset points", xytext=(0,10), ha='center')

plt.title('Visualización de Embeddings con t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.grid(True)
plt.show()


**Ejercicio**

Extiende el código para guardar la visualización como una imagen o explorar otros métodos de reducción de dimensionalidad, como PCA o UMAP.

In [ ]:
## Tu respuesta